# 2. Motor de Otimização (Validacao)

**Objetivo:** Validar a lógica de slotting utilizando os módulos Python.


In [ ]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath('.'))

from src import slotting_engine

# Carregar dados gerados anteriormente
try:
    df_layout = pd.read_csv('data/layout_fisico.csv')
    df_skus = pd.read_csv('data/mestre_skus.csv')
    df_orders = pd.read_csv('data/pedidos_backlog.csv')
    print("Dados carregados de 'data/'.")
except FileNotFoundError:
    print("Arquivos não encontrados em 'data/'. Gerando novos...")
    # Fallback ou erro

# 1. Calcular Scores
sku_scores = slotting_engine.calculate_sku_scores(df_orders, df_skus)
print("Top 5 SKUs:")
print(sku_scores.head())

# 2. Calcular Custos de Bins
df_layout_costed = slotting_engine.calculate_bin_costs(df_layout)
print("Top 5 Bins:")
print(df_layout_costed.head())

# 3. Alocar (Greedy)
df_alloc = slotting_engine.run_greedy_allocation(sku_scores, df_layout_costed)
df_alloc.to_csv('data/mapa_alocacao_inicial.csv', index=False)
print(f"Alocação Inicial concluída: {len(df_alloc)} SKUs.")

# 4. Testar Otimização (Hill Climbing)
print("\nIniciando Otimização Hill Climbing (Teste Rápido)...")
df_alloc_opt, history = slotting_engine.optimize_slotting_hill_climbing(
    df_alloc, df_orders, df_layout, iterations=10, sample_size=10
)
print(f"Otimização concluída. Histórico: {len(history)} iterações.")
df_alloc_opt.to_csv('data/mapa_alocacao_otimizado.csv', index=False)